In [1]:
from tabnanny import check
from matplotlib.image import imread
import numpy as np
from numpy import imag
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread,imshow
from skimage.transform import resize
import xlsxwriter
import mss
import mss.tools
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
from operator import indexOf

# GUI

In [2]:
class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.x = self.y = 0
        
        self.canvas = tk.Canvas(self, width=300, height=300, bg = "white", cursor="cross")
        self.label = tk.Label(self, text="Draw the letter(A or J) for now you want to recognize with genetics algorithms \n then press the take screen button to save the image \n then close this window", font=("Helvetica", 20))
        self.label1 = tk.Label(self, text="please first maximize this window before captureing the image:", font=("Helvetica", 30))
        self.takeScreen_btn = tk.Button(self, text = "takeScreen", command = self.takeScreen)   
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)
        # Grid structure
        self.canvas.grid(row=6, column=0, pady=2, sticky=W, )
        self.label.grid(row=6, column=1,pady=2, padx=2)
        self.takeScreen_btn.grid(row=8, column=1, pady=2, padx=2)
        self.button_clear.grid(row=8, column=0, pady=2)
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label1.grid(row=0, column=1,pady=2, padx=2)

        self.canvas.bind("<B1-Motion>", self.draw_lines)
        
    def clear_all(self):
        self.canvas.delete("all")
        
    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')

    def takeScreen(self):
        with mss.mss() as sct:
            monitor = {"top": 25, "left": 0, "width": 300, "height": 300}
            output = "checkImage.png".format(**monitor)
            sct_img = sct.grab(monitor)
            mss.tools.to_png(sct_img.rgb, sct_img.size, output=output)
            print(output)

In [3]:
def fuction_to_capture_and_store_the_check_image():
    app = App()
    mainloop()

# Initialization

In [4]:
def function_to_extract_the_features_of_the_initial_population_for_J_dataset():
    data = pd.read_excel("j.xlsx")
    print("-------------------------Column headings----------------------------------------")
    print(list(data))
    print("-----------------------------------------------------------------")
    images=data['image path']
    list_of_images_graphs=[]
    print("--------------------------extracting the verteces of each image in the dataset seperatly---------------------------------------")
    for i in range(54):
        print("extracting the features grapph for image number:", i," in the dataset")
        readed_image=imread(images[i])
        features_matrix=np.zeros((readed_image.shape[0],readed_image.shape[1]))
        for i in range(0,readed_image.shape[0]):
            for j in range(0,readed_image.shape[1]):
                features_matrix[i][j] = ((int(readed_image[i,j,0]) + int(readed_image[i,j,1]) + int(readed_image[i,j,2]))/3)
        resized_image=resize(features_matrix,(30,30))

        garaph_of_the_current_image=[]
        for i in range(0,resized_image.shape[0]):
            for j in range(0,resized_image.shape[1]):
                if(resized_image[i][j]<=150):
                    garaph_of_the_current_image.append((i,j))
        list_of_images_graphs.append(garaph_of_the_current_image)    
    return list_of_images_graphs

In [5]:
def function_to_extract_the_features_of_the_initial_population_For_a_dataset():
    data = pd.read_excel("a.xlsx")
    print("-------------------------Column headings----------------------------------------")
    print(list(data))
    print("-----------------------------------------------------------------")
    images=data['image path']
    list_of_images_graphs=[]
    print("--------------------------extracting the verteces of each image in the dataset seperatly---------------------------------------")
    for i in range(54):
        print("extracting the features grapph for image number:", i," in the dataset")
        readed_image=imread(images[i])
        features_matrix=np.zeros((readed_image.shape[0],readed_image.shape[1]))
        
        for i in range(0,readed_image.shape[0]):
            for j in range(0,readed_image.shape[1]):
                features_matrix[i][j] = ((int(readed_image[i,j,0]) + int(readed_image[i,j,1]) + int(readed_image[i,j,2]))/3)
        resized_image=resize(features_matrix,(30,30))

        garaph_of_the_current_image=[]
        for i in range(0,resized_image.shape[0]):
            for j in range(0,resized_image.shape[1]):
                if(resized_image[i][j]<=150):
                    garaph_of_the_current_image.append((i,j))
        #adding the graph of the curent image in the list 
        list_of_images_graphs.append(garaph_of_the_current_image)    
    return list_of_images_graphs

In [6]:
def function_to_extract_the_black_pixels_of_check_image():
    readed_image=imread("checkImage.png",as_gray=False)
    features_matrix=np.zeros((readed_image.shape[0],readed_image.shape[1]))

    for i in range(0,readed_image.shape[0]):
        for j in range(0,readed_image.shape[1]):
            features_matrix[i][j] = ((int(readed_image[i,j,0]) + int(readed_image[i,j,1]) + int(readed_image[i,j,2]))/3)
    resized_image=resize(features_matrix,(30,30))
    
    garaph_of_the_current_image=[]
    for i in range(0,resized_image.shape[0]):
        for j in range(0,resized_image.shape[1]):
            if(resized_image[i][j]<=150):
                garaph_of_the_current_image.append((i,j))
    return garaph_of_the_current_image

# Fitness Function

In [7]:
def fitness(pic1,pic2):
    resultt=[]
    small_list=len(pic1)<len(pic2) and pic1 or pic2
    big_list=len(pic1)>len(pic2) and pic1 or pic2
    for i in range(0, len(small_list)):
        resultt.append((abs(pic1[i][0]-pic2[i][0]),abs(pic1[i][1]-pic2[i][1])))
    temp=big_list[len(small_list):]
    resultt.extend(temp)
    sum=0
    for i in range(0, len(resultt)):
        sum+=resultt[i][0]+resultt[i][0]
    return(sum)

In [8]:
def clculate_fitness_of_checkImageAndTheinitialPopulation(initialPopulation,checkImage):
    resul=[]
    for i in range(len(initialPopulation)):
        temp=initialPopulation[i]
        resul.append(fitness(temp[:],checkImage))
    return(resul)

# Selection

In [9]:
def TheNFitestIndexOfCromosomes(fitnessArray,numberOfTheNeededCromosomes):
    temp=fitnessArray[:]
    fitestN=[]
    fitestIndexs=[]
    n=0
    for i in range(numberOfTheNeededCromosomes):
        a=min(temp)
        fitestN.append(a)
        temp.pop(indexOf(temp,a))
    for i in range(numberOfTheNeededCromosomes):
        index=indexOf(fitnessArray,fitestN[i])
        fitestIndexs.append(index)
    return(fitestIndexs)

In [10]:
def convertTheIndexToCromosom(arrayOfIndexes,initialPopulation):
    ar=[]
    for i in range(len(arrayOfIndexes)):
        ar.append(initialPopulation[arrayOfIndexes[i]])
    return(ar)

# Cross-Over

In [11]:
def crossPoint(paran1,paran2):
    small_list=len(paran1)<len(paran2) and paran1 or paran2
    return(len(small_list)//2)

In [12]:
def crossoverForTwoLists(paran1,paran2):
    crosPoint=crossPoint(paran1,paran2)
    tempx=paran1[:]
    temp=paran1[0:crosPoint]
    temp1=paran2[crosPoint:]
    temp.extend(temp1)
    arr=temp[:]
    temp=[]
    temp1=[]
    temp=paran2[0:crosPoint]
    temp1=tempx[crosPoint:]
    temp.extend(temp1)
    paran2=temp[:]
    return(paran1,paran2)

In [13]:
def crossOver(listOfNFitestIndex,initialPopulation):
    listOfTheFitestChromosomes=convertTheIndexToCromosom(listOfNFitestIndex,initialPopulation)
    listOfOfsprings=[]
    for i in range(3):
        ofs1,ofs2=crossoverForTwoLists(listOfTheFitestChromosomes[i],listOfTheFitestChromosomes[i+1])
        listOfOfsprings.append(ofs1)
        listOfOfsprings.append(ofs2)
    listOfTheFitestChromosomes.extend(listOfOfsprings)
    return(listOfTheFitestChromosomes)

# Starting Point :)

In [14]:
print(" the start of excution ".center(100,'*'))
fuction_to_capture_and_store_the_check_image()
initialPopulationForA= function_to_extract_the_features_of_the_initial_population_For_a_dataset()
print("*"*100)
print(" the check image features vector ".center(100,'*'))
print("*"*100)
checkImage=function_to_extract_the_black_pixels_of_check_image()
print(" the fitness values for a are ".center(100,'*'))
fitListForA=clculate_fitness_of_checkImageAndTheinitialPopulation(initialPopulationForA,checkImage)
theFitestIthsForA=TheNFitestIndexOfCromosomes(fitListForA,5)
wasBEFORETHECROSSOVERForA=convertTheIndexToCromosom(theFitestIthsForA,fitListForA)
print("*"*100)
print("the fitness was :\n",wasBEFORETHECROSSOVERForA)
print("*"*100)
listOfNewPopulationForA=crossOver(theFitestIthsForA,initialPopulationForA)
newFitListForA=clculate_fitness_of_checkImageAndTheinitialPopulation(listOfNewPopulationForA,checkImage)
print("the fitness is now:\n",newFitListForA)
print(" the fitness of being A is ".center(100,'*'))
probabilityOfBiengA=1/min(newFitListForA)
print(probabilityOfBiengA)


initialPopulationForJDataSet= function_to_extract_the_features_of_the_initial_population_for_J_dataset()
print("*"*100)
print(" the fitness values for J are ".center(100,'*'))
fitListForJ=clculate_fitness_of_checkImageAndTheinitialPopulation(initialPopulationForJDataSet,checkImage)
theFitestIthsForJ=TheNFitestIndexOfCromosomes(fitListForJ,5)
wasBEFORETHECROSSOVErForJ=convertTheIndexToCromosom(theFitestIthsForJ,fitListForJ)
print("*"*100)
print("the fitness for J was :\n",wasBEFORETHECROSSOVErForJ)
print("*"*100)
listOfNewPopulationForj=crossOver(theFitestIthsForJ,initialPopulationForJDataSet)
newFitListForJ=clculate_fitness_of_checkImageAndTheinitialPopulation(listOfNewPopulationForj,checkImage)
print("the fitness for J is now:\n",newFitListForJ)
print(" the fitness of being J is ".center(100,'*'))
probabilityOfBiengj=1/min(newFitListForJ)
print(probabilityOfBiengj)

if (probabilityOfBiengA>probabilityOfBiengj):
    print("the handwrtten letter was :- A")
else:
    print("the handwrtten letter was :- j")

************************************** the start of excution ***************************************
checkImage.png
-------------------------Column headings----------------------------------------
['image path', 'character']
-----------------------------------------------------------------
--------------------------extracting the verteces of each image in the dataset seperatly---------------------------------------
extracting the features grapph for image number: 0  in the dataset
extracting the features grapph for image number: 1  in the dataset
extracting the features grapph for image number: 2  in the dataset
extracting the features grapph for image number: 3  in the dataset
extracting the features grapph for image number: 4  in the dataset
extracting the features grapph for image number: 5  in the dataset
extracting the features grapph for image number: 6  in the dataset
extracting the features grapph for image number: 7  in the dataset
extracting the features grapph for image numb

extracting the features grapph for image number: 44  in the dataset
extracting the features grapph for image number: 45  in the dataset
extracting the features grapph for image number: 46  in the dataset
extracting the features grapph for image number: 47  in the dataset
extracting the features grapph for image number: 48  in the dataset
extracting the features grapph for image number: 49  in the dataset
extracting the features grapph for image number: 50  in the dataset
extracting the features grapph for image number: 51  in the dataset
extracting the features grapph for image number: 52  in the dataset
extracting the features grapph for image number: 53  in the dataset
****************************************************************************************************
*********************************** the fitness values for J are ***********************************
****************************************************************************************************
the fitness for J